# Ejercicio Data Quality - Perfilado
## Evaluar la calidad de datos de las ventas de productos

Se quiere hacer una evaluación de calidad de datos sobre las ventas (sales) y pagos (payments). Para ello se requiere hacer un análisis de los siguientes puntos:
- Calidad de los datos
- Selección de clave principal
- Identificación de cardinalidad
- Obtener media, varianza y desviacion Estandar, covarianza, correlacion
- Mejorar la calidad.

**Referencia**: “Estadística Descriptiva con Python y Pandas”: https://coderhook.github.io/Descriptive%20Statistics

- Columnas sales:, orderNumber, orderLineNumber, orderDate, shippedDate, requiredDate, customerNumber, employeeNumber, productCode, status, comments, quantityOrdered, priceEach, sales_amount, origin

- Columnas payments:, customerNumber, checkNumber, paymentDate, amount

In [19]:
import pandas as pd
import numpy as np

In [20]:
# Leer el archivo CSV desde la URL
url = "https://raw.githubusercontent.com/ricardoahumada/DataScienceBasics/refs/heads/main/data/company_sales/sales.csv"
sales_df = pd.read_csv(url)

In [21]:
# Mostrar las primeras filas del DataFrame para entender su estructura
sales_df.columns = [
    'orderNumber', 'orderLineNumber', 'orderDate', 'shippedDate', 'requiredDate', 
    'customerNumber', 'employeeNumber', 'productCode', 'status', 'comments', 
    'quantityOrdered', 'priceEach', 'sales_amount', 'origin'
]
print("Primeras filas del DataFrame:")
print(sales_df.head())

Primeras filas del DataFrame:
   orderNumber  orderLineNumber   orderDate shippedDate requiredDate  \
0        10100                1  0000-00-00  0000-00-00   0000-00-00   
1        10100                2  0000-00-00  0000-00-00   0000-00-00   
2        10100                3  0000-00-00  0000-00-00   0000-00-00   
3        10100                4  0000-00-00  0000-00-00   0000-00-00   
4        10101                1  0000-00-00  0000-00-00   0000-00-00   

   customerNumber  employeeNumber productCode   status  \
0             363            1216    S24_3969  Shipped   
1             363            1216    S18_2248  Shipped   
2             363            1216    S18_1749  Shipped   
3             363            1216    S18_4409  Shipped   
4             128            1504    S18_2795  Shipped   

                 comments  quantityOrdered  priceEach  sales_amount origin  
0                     NaN               49      35.29       1729.21  spain  
1                     NaN         

In [28]:
#se ve que la columna de fechas 'orderDate'
#tiene valores nulos o erroneos. Podemos hacer un sample de 20 filas para asegurarnos
sales_df['orderDate'].sample(20)

161     0000-00-00
1864    0000-00-00
1949    0000-00-00
402     0000-00-00
272     0000-00-00
2500    0000-00-00
2291    0000-00-00
2274    0000-00-00
1158    0000-00-00
2518    0000-00-00
1029    0000-00-00
2028    0000-00-00
1141    0000-00-00
2529    0000-00-00
2576    0000-00-00
1868    0000-00-00
1134    0000-00-00
284     0000-00-00
605     0000-00-00
397     0000-00-00
Name: orderDate, dtype: object

In [29]:
#verificar con un unique pra ver los valeres únicos
sales_df['orderDate'].unique()
#vemos que existe algún valor que es distinto de 0, pero pocos

array(['0000-00-00', '2038-09-00'], dtype=object)

In [31]:
#por lo que hacemos un value count
sales_df['orderDate'].value_counts()
#vemos que sólo hay tres valores significativos

0000-00-00    2998
2038-09-00       3
Name: orderDate, dtype: int64

In [22]:
# Mostrar información general del DataFrame
print("\nInformación general del DataFrame:")
print(sales_df.info())


Información general del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3001 entries, 0 to 3000
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   orderNumber      3001 non-null   int64  
 1   orderLineNumber  3001 non-null   int64  
 2   orderDate        3001 non-null   object 
 3   shippedDate      2859 non-null   object 
 4   requiredDate     3001 non-null   object 
 5   customerNumber   3001 non-null   int64  
 6   employeeNumber   3001 non-null   int64  
 7   productCode      3001 non-null   object 
 8   status           3001 non-null   object 
 9   comments         759 non-null    object 
 10  quantityOrdered  3001 non-null   int64  
 11  priceEach        3001 non-null   float64
 12  sales_amount     3001 non-null   float64
 13  origin           3001 non-null   object 
dtypes: float64(2), int64(5), object(7)
memory usage: 328.4+ KB
None


In [23]:
# Mostrar estadísticas descriptivas del DataFrame
print("\nEstadísticas descriptivas del DataFrame:")
print(sales_df.describe())


Estadísticas descriptivas del DataFrame:
        orderNumber  orderLineNumber  customerNumber  employeeNumber  \
count   3001.000000      3001.000000     3001.000000     3001.000000   
mean   10260.509164         6.424525      259.639120     1317.948684   
std       92.619750         4.196870      118.403435      326.343575   
min    10100.000000         1.000000      103.000000        0.000000   
25%    10181.000000         3.000000      145.000000     1216.000000   
50%    10263.000000         6.000000      240.000000     1370.000000   
75%    10339.000000         9.000000      353.000000     1501.000000   
max    10425.000000        18.000000      496.000000     1702.000000   

       quantityOrdered    priceEach  sales_amount  
count      3001.000000  3001.000000   3001.000000  
mean         35.211929    90.765831   3204.908437  
std           9.828957    36.579368   1631.356967  
min           6.000000    26.550000    481.500000  
25%          27.000000    62.000000   1988.700000

In [24]:
# Verificar la cantidad de valores nulos en cada columna
print("\nCantidad de valores nulos en cada columna:")
print(sales_df.isnull().sum())


Cantidad de valores nulos en cada columna:
orderNumber           0
orderLineNumber       0
orderDate             0
shippedDate         142
requiredDate          0
customerNumber        0
employeeNumber        0
productCode           0
status                0
comments           2242
quantityOrdered       0
priceEach             0
sales_amount          0
origin                0
dtype: int64


In [25]:
# Verificar la cantidad de valores únicos en cada columna
print("\nCantidad de valores únicos en cada columna:")
print(sales_df.nunique())


Cantidad de valores únicos en cada columna:
orderNumber         326
orderLineNumber      18
orderDate             2
shippedDate           3
requiredDate          3
customerNumber       98
employeeNumber       15
productCode         109
status                6
comments             37
quantityOrdered      61
priceEach          1573
sales_amount       2885
origin                2
dtype: int64


In [26]:
# Mostrar la distribución de valores en cada columna categórica
print("\nDistribución de valores en columnas categóricas:")
for column in sales_df.select_dtypes(include=['object']).columns:
    print(f"\nDistribución de la columna '{column}':")
    print(sales_df[column].value_counts())
    


Distribución de valores en columnas categóricas:

Distribución de la columna 'orderDate':
0000-00-00    2998
2038-09-00       3
Name: orderDate, dtype: int64

Distribución de la columna 'shippedDate':
0000-00-00    2839
2038-00-06      17
2038-09-07       3
Name: shippedDate, dtype: int64

Distribución de la columna 'requiredDate':
0000-00-00    2981
2038-00-08      17
2038-09-07       3
Name: requiredDate, dtype: int64

Distribución de la columna 'productCode':
S18_3232    53
S18_2238    29
S18_3136    29
S24_2000    28
S24_4278    28
            ..
S24_3432    25
S24_1628    25
S24_3969    25
S18_4933    24
S24_2887    24
Name: productCode, Length: 109, dtype: int64

Distribución de la columna 'status':
Shipped       2775
Cancelled       79
Resolved        47
On Hold         44
In Process      42
Disputed        14
Name: status, dtype: int64

Distribución de la columna 'comments':
They want to reevaluate their terms agreement with Finance.                                            